# 1. 
Comme $M$ est une matrice symmétrique définie positive, ses valeurs propres seront toutes réelles et positives et égales à ses valeurs singulières. Les valeurs propres de $M$ respecteront $Mv= cv$ pour une valeur propre $c$ réelle positive. Si on cherche maintenant les valeurs propres de $M + \lambda² I$, on peut voir que
$$
\begin{align}
 (M + \lambda² I)v &= dv\\
  Mv + \lambda²v &= dv\\
  Mv &= dv - \lambda²v\\
  cv &= dv - \lambda²v\\
  d &= c + \lambda²
\end{align}
$$

Donc les valeurs propres (et singulières) de $M + \lambda² I$ sont celles de $M$ mais décalé de $\lambda²$. De cela, si le nombre de conditionnement de $M$ est 
$$ \kappa(M) = \sigma_\text{max}(M) / \sigma_\text{min}(M) $$

alors 
$$ \kappa(M + \lambda²) = (\sigma_\text{max}(M) + \lambda²) / (\sigma_\text{min}(M)+ \lambda²)$$

Or comme $\lambda² > 0$, cela implique que $ \kappa(M) > \kappa(M + \lambda²)$.

# 2. 

# 3. 

# 4. 

# 5. 

# 6. 

In [ ]:
using LinearAlgebra
using SparseArrays
using Printf
using Test

In [ ]:
function test_least_square(f::Function, minimum_norm::Bool)
    A_1 = [1. 2. 3.; 4. 5. 6.; 7. 8. 9.; 5. 7. 9.]
    b_1 = ones(Float64, 4)
    
    x = f(sparse(A_1), b_1)
    x_baseline = A_1 \ b_1

    r = A_1 * x - b_1
    r_baseline = A_1 * x_baseline - b_1
    @test norm(r - r_baseline) ≈ 0.0 atol=1e-6   

    if minimum_norm
        @test norm(x - x_baseline) ≈ 0.0 atol=1e-6
    end
end

In [ ]:
function golub_riley(A::SparseMatrixCSC{Float64}, b::Vector{Float64}, λ::Float64=0.1, ϵ::Float64=1e-6)
    m, n = size(A)
    x_k = zeros(Float64, n)
    b_k = [b; zeros(n)]
    A_augmented = sparse([A; λ * I])
    QR_A_augmented = qr(A_augmented)

    while true
        Δx_k = QR_A_augmented \ b_k
        x_k += Δx_k
        b_k = [b - A * x_k; zeros(n)]
        if (norm(Δx_k) / norm(x_k)) ≤ ϵ
            break
        end
    end
    
    return x_k
end


In [ ]:
test_least_square(golub_riley, false)

# 7. 

# 8. 

In [ ]:
using HarwellRutherfordBoeing

In [ ]:
function get_problem_from_animal(path_to_animal_folder::String, problem_name::String)
    A = HarwellBoeingMatrix(joinpath(path_to_animal_folder, "hb", problem_name * ".hb"))
    return A.matrix, vec(A.rhs)
end

In [ ]:
@enum animal_problem begin
    small
    small2
    large
    large2
    medium
    medium2
    very
    very2
end

# 9. 

In [ ]:
function scale_columns_to_unit_norm!(A::SparseMatrixCSC{Float64})
    for j = 1:A.n
        #normalize!(A[:, j])
        col_norm = norm(A[:, j])
        A[:, j] ./= col_norm
    end
end

In [ ]:
# Tests
for n = 10:30
    A = sprandn(Float64, n, n, 0.5)
    scale_columns_to_unit_norm!(A)
    for j = 1:n
        @test norm(A[:, j]) ≈ 1.0
    end
end

# 10. 

# 11. 

# 12. 

On peut se servir du fait que deux matrices congruentes ont la même intertie. Soit 
$$ K = 
\begin{pmatrix}
I & A \\
A^T & - \lambda^{2}I
\end{pmatrix}
$$

On cherche alors une matrice $D$ congruente à la matrice bloc $K$, ce qui veut dire que $K = M^*DM$ où $M$ est inversible. Regardons
\begin{equation*}
  K = M^* D M =
  \left(
    \begin{array}{cc}
    I & 0 \\
    A^* & I
    \end{array}
  \right)
  \left(
  \begin{array}{cc}
    I & 0 \\
    0 & -A^* A - \lambda^{2} I 
  \end{array}
\right)
\left(
  \begin{array}{cc}
    I & A \\
    0 & I        
  \end{array}
  \right)
\end{equation*}

On calcule aisément que cette égalité tient. On voit également que $M$ et $M^*$ sont inversibles car elles ont un déterminant de $1$ car elles n'ont que des éléments de $1$ sur la diagonale. $K$ est donc congruente à $D$.

Il faut maintenant examiner les valeurs propres de $D$. Par sa structure de bloc uniquement sur la diagonale, les valeurs propres de $D$ seront l'union des valeurs propres de $I$ et de $-A^*A - \lambda^{2}I$. Les valeurs propres de $I$ sont $1$. 
Quant à $-A^*A - \lambda^{2}I$, regardons pour un vecteur $x$
\begin{align*}
x^*(-A^*A - \lambda^{2}I)x &= -x^* A^*A x - \lambda^{2}x^*x\\
&= -||Ax||^{2} - ||x||^{2}
\end{align*}

La norme de $||Ax||$ pourrait être nulle pour un vecteur $x$ non-nulle, mais dans ce cas la norme de $x$ serait positive et donc le membre de droite de l'équation serait négatif. On en conclut que le membre de droite est nulle seulement si $x$ est nul et donc que le bloc en bas à droite de $D$ est défini négatif, et donc que $D$ a seulement des valeurs propres strictement négative ou positive.

Comme $K$ a la même inertie que $D$, elle aura des valeurs propres strictement positives et strictement négatives, ce qui veut dire que la matrice est inversible.


# 13. 

In [ ]:
using LDLFactorizations

In [ ]:
function golub_riley_2(A::SparseMatrixCSC{Float64}, b::Vector{Float64}, λ::Float64=0.1, ϵ::Float64=1e-6)
    m, n = size(A)
    x_k = zeros(Float64, n)
    b_augmented_k = [b; zeros(n)]

    # We could pass an upper triangular matrix to ldl for more memory efficiency
    K = [I A; adjoint(A) (-(λ^2) * I)]
    println(size(K))
    LDLT = ldl(K)

    while true
        Δsol_k = LDLT \ b_augmented_k
        Δr_k = Δsol_k[1:m]
        Δx_k = Δsol_k[m+1:m+n]
        x_k += Δx_k
        b_augmented_k[1:m] = b - A * x_k
        if (norm(Δx_k) / norm(x_k)) ≤ ϵ
            break
        end
    end
    
    return x_k
end

In [ ]:
test_least_square(golub_riley_2, false)

# 14. 

# 15. 